In [10]:
from azure.storage.blob import BlobServiceClient
import json
import os
from PIL import Image
import matplotlib.pyplot as plt
import io  # Used to convert bytes to a file-like object
import cv2
import numpy as np


#set up storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=mlfinalexam5505462853;AccountKey=0c40lghglG5/GlNK9yujDQAgo38GKoS2I3DeC/g22hwAEIFANKpmC/TqOpRk4RCT1DbfNiHBFt72+AStB+PfUA==;EndpointSuffix=core.windows.net"
container_name = "publicdata"

#create client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

In [11]:
# Variables to accumulate dimensions
total_images = 0
total_width = 0
total_height = 0

# Loop through all blobs and accumulate dimensions
blobs = container_client.list_blobs()
for blob in blobs:
    if blob.name.endswith(".jpg"):
        # Download the image
        image_blob_client = container_client.get_blob_client(blob.name)
        image_content = image_blob_client.download_blob().readall()

        # Decode image with OpenCV
        image = cv2.imdecode(np.frombuffer(image_content, np.uint8), cv2.IMREAD_COLOR)

        # Accumulate dimensions
        height, width, _ = image.shape
        total_images += 1
        total_width += width
        total_height += height

# Calculate summary statistics
average_width = total_width / total_images
average_height = total_height / total_images

print(f"Total images: {total_images}")
print(f"Average width: {average_width}")
print(f"Average height: {average_height}")

Total images: 51
Average width: 1574.9019607843138
Average height: 885.8823529411765


In [8]:
# Parameters for preprocessing
desired_width = 64  # Desired width for resizing
desired_height = 64  # Desired height for resizing
median_kernel_size = (5, 5)  # Kernel size for median blur
canny_threshold1 = 0.4  # Lower threshold for Canny edge detection
canny_threshold2 = 0.8  # Upper threshold for Canny edge detection

# Loop through all blobs and process images directly
blobs = container_client.list_blobs()
for blob in blobs:
    if blob.name.endswith(".jpg"):
        # Download the image
        image_blob_client = container_client.get_blob_client(blob.name)
        image_content = image_blob_client.download_blob().readall()

        # Preprocess the image
        nparr = np.frombuffer(image_content, np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)  # Convert bytes to numpy array
        # Resize the image
        resized_image = cv2.resize(image, (desired_width, desired_height))

        # Normalize pixel values to [0, 1]
        normalized_image = resized_image / 255.0

        # Apply histogram equalization for contrast enhancement
        equalized_image = cv2.equalizeHist(normalized_image)

        # Apply noise reduction filters
        blurred_image = cv2.medianBlur(normalized_image, median_kernel_size)

        # Apply edge detection using Canny
        edges = cv2.Canny(blurred_image, canny_threshold1, canny_threshold2)

        # Save preprocessed image to a temporary file
        temp_file_path = "temp_image.jpg"
        cv2.imwrite(temp_file_path, edges)  # Convert normalized image

        # Upload the preprocessed image back to Azure Blob Storage
        with open(temp_file_path, "rb") as data:
            container_client.upload_blob(name="preprocessed/" + blob.name, data=data)

        # Clean up temporary file
        os.remove(temp_file_path)
